In [1]:
import cv2
import datetime
import pandas as pd
import numpy as np
import dlib
import os
from matplotlib import pyplot as plt
import skvideo.io
import matplotlib as mpl
import time

In [2]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [3]:
## face detector와 landmark predictor 정의
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
num = int(input('운동 횟수를 입력해주세요'))

In [5]:
webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("Could not open webcam")
    exit()

sample_num = 0
count = 0

while webcam.isOpened():
    status, frame = webcam.read()

    if status:
        cv2.imshow("test", frame)

    # q를 누르면 종료
    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        break

    # 자동으로 캡쳐
    sample_num = sample_num + 1

    if sample_num == 10:
    #if key & 0xFF == ord('s'):

        for i2 in range(num):
            count = 0
            df_x = pd.DataFrame(columns=['x' + str(x) for x in range(0, 68)]) # 좌표를 column으로
            df_y = pd.DataFrame(columns=['y' + str(x) for x in range(0, 68)])

            # cv2 이미지처리
            img = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            # resize
            r = 400. / img.shape[1]
            dim = (400, int(img.shape[0] * r))
            resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)

            # 얼굴 detection
            rects = detector(resized, 1)
            
            for i, rect in enumerate(rects):
                # 찾은 얼굴의 박스좌표
                l = rect.left()
                t = rect.top()
                b = rect.bottom()
                r = rect.right()

                # facial landmark 찾기
                shape = predictor(resized, rect)
                # facial landmakr를 빨간색 점으로 찍어서 표현
                x_list = []
                y_list = []
                for j in range(68):
                    x, y = shape.part(j).x, shape.part(j).y
                    x_list.append(x)
                    y_list.append(y)
                    cv2.circle(resized, (x, y), 1, (0, 0, 255), -1)
                    cv2.rectangle(resized, (l, t), (r, b), (0, 255, 0), 2)
                    result = cv2.putText(resized, str(j), (x, y), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 0.3, (0, 255, 0))
                
                # 얼굴이 있는 부분을 박스
                # 행추가를 좀 하고 싶다
                count += 1
                df_x.loc[count] = x_list # row 추가
                df_y.loc[count] = y_list

                createFolder('test_' + str(i2))
                df_x.to_csv('test_' + str(i2) + '\\test_dfx.csv')
                df_y.to_csv('test_' + str(i2) + '\\test_dfy.csv')

                # 이미지 저장할 경로 입력
                file = 'test_' + str(i2) + '\\' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S%f") + '.jpg'
                cv2.imwrite(file, result)

            del df_x
            del df_y

        print('finish') # 모두 저장
        sample_num = 0

webcam.release()
cv2.destroyAllWindows()

NameError: name 'df_x' is not defined

: 